This notebook is heavily inspired by [link](https://www.youtube.com/watch?v=aywZrzNaKjs&ab_channel=Rabbitmetrics)


In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

#### LLM WRAPPER

In [2]:
from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003")
llm("explain large language models in one sentence")

'\n\nLarge language models are neural networks trained on large datasets of natural language text, used to generate predictions about the probability of a given word or phrase appearing in a sentence.'

In [3]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [4]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)
messages = [
    SystemMessage(content="You are an expert data scientist"),
    HumanMessage(content="Write a python script that does sagemaker batch processing inside lambda function")
]
response=chat(messages)

In [5]:
print(response.content, end='\n')

Here is an example Python script that does SageMaker batch processing inside a Lambda function:

```python
import boto3

def lambda_handler(event, context):
    # Set up the SageMaker client
    sagemaker = boto3.client('sagemaker')

    # Set up the batch processing job parameters
    job_name = 'my-batch-processing-job'
    input_location = 's3://my-batch-processing-input-bucket/input/'
    output_location = 's3://my-batch-processing-output-bucket/output/'
    job_definition = 'my-batch-processing-job-definition'
    job_queue = 'my-batch-processing-job-queue'

    # Start the batch processing job
    response = sagemaker.create_processing_job(
        ProcessingInputs=[
            {
                'InputName': 'input',
                'S3Input': {
                    'S3Uri': input_location,
                    'LocalPath': '/opt/ml/processing/input',
                    'S3DataType': 'S3Prefix',
                    'S3InputMode': 'File',
                    'S3DataDistributionTyp

#### PROMT TEMPLATES

In [6]:
from langchain import PromptTemplate

template = """
You are an expert data scientist with an expetise in building ML model in AWS ecosystem.
Explain the concept of {concept} in a couple of lines
"""

promt = PromptTemplate(
    input_variables=["concept"],
    template=template
)

In [7]:
promt

PromptTemplate(input_variables=['concept'], output_parser=None, partial_variables={}, template='\nYou are an expert data scientist with an expetise in building ML model in AWS ecosystem.\nExplain the concept of {concept} in a couple of lines\n', template_format='f-string', validate_template=True)

In [8]:
llm(promt.format(concept="batch transform"))

'\nBatch Transform is a feature of AWS that allows you to run an ML model on a batch of data. It enables you to quickly and efficiently get predictions from an ML model on large amounts of data without having to manually set up the environment. You simply upload your data, point to the ML model, and the batch transform job will run in the background.'

In [9]:
# HERE we dynamically change the promt with user input
llm(promt.format(concept="model registry"))

'\nModel registry is a concept that allows for the tracking and managing of ML models throughout their entire lifecycles. It provides a centralized repository for tracking models, their data sources, their performance metrics, and the data and code used to create them. Additionally, it enables versioning and governance of ML models, making it easier to understand, replicate, and reuse models.'

#### CHAIN
It takes LLM and Template and combines them into an interface that takes input from the user and returnds output from LLM

In [10]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=promt)

# Run the chain only specidying the input vatiable
print(chain.run("autoencoder"))


An autoencoder is a type of artificial neural network used to learn efficient data representations in an unsupervised manner. It consists of an encoder which compresses data into a low-dimensional representation, and a decoder which reconstructs the input data from the compressed representation. Autoencoders are commonly used for dimensionality reduction, denoising, and feature extraction.


In [11]:
# the second chain takes the output of the first chain as input and does an action on it (in this case it 
# takes a concept from first chain and explains it to me like I'm fine years old)

second_promt = PromptTemplate(
    input_variables=["ml_concept"],
    template="Trun the concept description of {ml_concept} and explain it to me like I am five in 100 words"
)
chain_two = LLMChain(llm=llm, prompt=second_promt)

In [12]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

# Run the chain specifying only the input variable for the first chain
explanation = overall_chain.run("autoencoder")
print(explanation)



> Entering new SimpleSequentialChain chain...

Autoencoders are unsupervised learning algorithms that attempt to reconstruct their input. They are used for dimensionality reduction, feature extraction, and representation learning. Autoencoders consist of two parts: an encoder and a decoder. The encoder compresses the input into a latent space representation, while the decoder reconstructs the input from the latent space representation.


Autoencoders are like machines that take something in and do something to make it smaller. They do this by making a copy of the thing they take in and then changing it so that it takes up less space. The machine then remembers how it changed the thing and it uses that information to put it back together again. It's like taking a jigsaw puzzle apart and then putting it back together again, but without ever seeing the picture on the box. Autoencoders are useful for making things smaller so it takes up less space, like shrinking a photo or making a long

#### Embedding and VectorStores
Split the explanation into chunks to be stored in VectorStore - "pincone"

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 50,
    chunk_overlap = 0,
)
text = text_splitter.create_documents([explanation])

In [14]:
text

[Document(page_content='Autoencoders are like machines that take something', metadata={}),
 Document(page_content='in and do something to make it smaller. They do', metadata={}),
 Document(page_content='this by making a copy of the thing they take in', metadata={}),
 Document(page_content='and then changing it so that it takes up less', metadata={}),
 Document(page_content='space. The machine then remembers how it changed', metadata={}),
 Document(page_content='the thing and it uses that information to put it', metadata={}),
 Document(page_content="back together again. It's like taking a jigsaw", metadata={}),
 Document(page_content='puzzle apart and then putting it back together', metadata={}),
 Document(page_content='again, but without ever seeing the picture on the', metadata={}),
 Document(page_content='box. Autoencoders are useful for making things', metadata={}),
 Document(page_content='smaller so it takes up less space, like shrinking', metadata={}),
 Document(page_content='a ph

In [15]:
text[0].page_content

'Autoencoders are like machines that take something'

In [16]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model_name="ada")

In [17]:
# vector representation of 0s embedding
query_results = embeddings.embed_query(text[0].page_content)
print(query_results[:5])
print(len(query_results))

[-0.02128082520253445, 0.029654279441033268, 0.02945004785038224, 0.014173601796930103, 0.013928525192000475]
1024


In [18]:
import os
import pinecone
from langchain.vectorstores import Pinecone

pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),
    environment=os.getenv('PINECONE_ENV')
)

/opt/anaconda3/envs/langchain-sandbox/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [19]:
print(pinecone)

<module 'pinecone' from '/opt/anaconda3/envs/langchain-sandbox/lib/python3.11/site-packages/pinecone/__init__.py'>


In [20]:
# we need to create an index first; use the lenght of 0s embedding as a dimension
dimension = len(query_results)
index_name = "langchain-quickstart"
pinecone.create_index(index_name, dimension=dimension)

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=UTF-8', 'date': 'Sat, 20 May 2023 21:10:35 GMT', 'x-envoy-upstream-service-time': '706', 'content-length': '131', 'server': 'envoy'})
HTTP response body: The index exceeds the project quota of 1 pods by 1 pods. Upgrade your account or change the project settings to increase the quota.


In [21]:
search = Pinecone.from_documents(text, embeddings, index_name=index_name)

In [22]:
query = "What is magical about an autoencoder?"
result = search.similarity_search(query)

In [23]:
# All the relevant chunks are extracted
result

[Document(page_content='An autoencoder is a type of machine learning. It', metadata={}),
 Document(page_content="data, which is how the autoencoder learns. It's a", metadata={}),
 Document(page_content='box. Autoencoders are useful for making things', metadata={}),
 Document(page_content='Autoencoders are like machines that take something', metadata={})]

#### AGENT

In [24]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI

In [25]:
agent_executor = create_python_agent(
    llm=OpenAI(temperature=0, max_tokens=500),
    tool=PythonREPLTool(),
    verbose=True
)

In [26]:
# This allows LLM to run python code
agent_executor.run("Find the roots (zeros) if the quadratic function 3 * x**2 + 2*x - 1")



> Entering new AgentExecutor chain...
 I need to solve a quadratic equation
Action: Python REPL
Action Input: import numpy as np
Observation: 
Thought: I can use the numpy function to solve the equation
Action: Python REPL
Action Input: np.roots([3, 2, -1])
Observation: 
Thought: I now know the final answer
Final Answer: (-1.0, 0.3333333333333333)

> Finished chain.


'(-1.0, 0.3333333333333333)'

In [27]:
# remove the index since free version of Pinecode allows only one index to be stored
index_name = "langchain-quickstart"
pinecone.delete_index(index_name)